In [1]:
import urllib.parse
import urllib.request
import pandas as pd
import pickle
import re
import numpy as np

In [2]:
def NotBruteAtAll(temp):
    res = []
    for t in temp:
        result = [] #ответ в шаблоне ФИО
        t = t.lower()
        words = t.split(" ")
        
        resultStrict = StrictCheck(words)
        order = [None, None, None]
        check = СheckMatrix(resultStrict)
        if check:
            order = GetOrder(resultStrict)
            for i in range(3):
                result.append(words[order[i]])   
            result = [words[order[0]],words[order[1]],words[order[2]]]
        # + установить, что если частота меньше N проверить реплейсом (т.к. только редкие, э-экономия)
        elif not check or any(i < 10 for i in resultStrict):
            resultReplWords, resultReplValues = ReplaceCheck(words)
            resultGramma = GrammaCheck(words) 
            replaced = np.array(resultReplValues) + np.array(resultGramma)+ np.array(statistics)
            strict = np.array(resultStrict) + np.array(resultGramma)+ np.array(statistics)
            
            strictOrder, stictWords, strictValues = GetMaxWordsValues(strict,[words]*3)
            replacedOrder, replacedWords, replacedValues = GetMaxWordsValues(replaced,resultReplWords)
            
            for i in range(3):
                if (strictValues[strictOrder[i]]>replacedValues[replacedOrder[i]]):
                    result.append(stictWords[strictOrder[i]])
                    order[i] = strictOrder[i]
                else:
                    result.append(replacedWords[replacedOrder[i]])
                    order[i] = replacedOrder[i]
        SetStatistics(order)
        result = result[0].title() + " " + result[1].title() + " " + result[2].title()
        res.append(result)
    return res

In [3]:
def GetOrder(matrix):
    order = [0,0,0]
    for j in range(3):
        for i in range(3):
            if (matrix[i][j]!=0):
                order[j] = i
                break
    return order

In [5]:
def GetOrderAndCheck(matrix):
    order = [None,None,None]
    for j in range(3):
        for i in range(3):
            if (matrix[i][j]!=0):
                if (i not in order):
                    order[j] = i
                    break
                else: return False
        if order[j] == None: return False
    return order

In [6]:
def СheckMatrix(matrix): #Если у нас в одной строке и столбце только по одному элементу, то возвращает True
    for i in range(3):
        k1 = 0
        if (matrix[i] != 0):
            k1 += 1
            k2 = 0
            for j in range(3):
                if (matrix[i][j] != 0): k2+=1
            if (k2 != 1):
                return False
        if (k1 != 1):
            return False
    return True

In [8]:
def GetMaxOrder(matrix):
    order = [None,None,None]
    for j in range(3):
        maxI = 0
        for i in range(3):
            if (matrix[i][j]>matrix[maxI][j]):
                maxI = i
        if (maxI not in order):
            order[j] = maxI
        else: return False
    return order

In [9]:
def GetMaxWords(matrix, matrixWords):
    order = [None,None,None]
    words = [None,None,None]
    for j in range(3):
        maxI = 0
        for i in range(3):
            if (matrix[i][j]>matrix[maxI][j]):
                maxI = i
        if (maxI not in order):
            order[j] = maxI
            words[j] = matrixWords[maxI][j]
        else: return False
    return order, words


In [10]:
def GetMaxWordsValues(matrixValues, matrixWords):
    order = [None,None,None]
    words = [None,None,None]
    values = [0,0,0]
    for j in range(3):
        maxI = 0
        for i in range(3):
            if (matrixValues[i][j]>matrixValues[maxI][j]):
                maxI = i
        if (maxI not in order):
            order[j] = maxI
            words[j] = matrixWords[maxI][j]
            values[j] = matrixValues[maxI][j]
        else: return False
    return order, words, values

In [11]:
def StrictCheck(words): #строгая проверка
    #          F I O
    result = [[0,0,0],
              [0,0,0],
              [0,0,0]]
    for i in range(3):
        w = words[i].strip()
        if (w in all_surnames): 
            result[i][0] = all_surnames[w]    
        if (w in all_names):
            result[i][1] = all_names[w]          
        if (w in all_patronymics):
            result[i][2] = all_patronymics[w] 
    return result

In [12]:
def ReplaceCheck(words): #проверка с перестановками
    #          F I O
    resultWords = [[0,0,0],
                  [0,0,0],
                  [0,0,0]]
    resultValues = [[0,0,0],
                  [0,0,0],
                  [0,0,0]]
    for i in range(3):
        w = words[i].strip()
        resultWords[i][0], resultValues[i][0] = forReplaceCheck(w, all_surnames)
        resultWords[i][1], resultValues[i][1] = forReplaceCheck(w, all_names) 
        resultWords[i][2], resultValues[i][2] = forReplaceCheck(w, all_patronymics)
    return resultWords, resultValues

In [13]:
def forReplaceCheck(w, list): #отдельный метод, составляет списки слов с 1 и 2 расстоянием
    res = [{"": 0},{"": 0}]
    for l in list:
        dist = shortDistance(w, l)
        if (dist != False):
            res[int(dist)-1][l] = list[l]*10**(-1*dist)

    key1, value1 = max(res[0].items(), key=lambda x:x[1])
    key2, value2 = max(res[1].items(), key=lambda x:x[1])
    
    if (value1>value2):
        return key1, value1
    else: return key2, value2

In [14]:
def shortDistance(word, dictWord): #упрощенное расстояние (поиск только с опечаткой 1 и 2)
    #word - слово которое проверяем
    #dictWord - слово из списка, с которым сравниваем
    if word == dictWord: return 0
    elif len(word) == 0: return len(dictWord)
    elif len(dictWord) == 0: return len(word)
    elif len(dictWord) > len(word)+2 and len(dictWord) < len(word)+2:
        return False
    #dist = levenshtein(word, dictWord)
    dist = damerau(word, dictWord)
    if (dist == 1 or dist==2):
        return dist
    else: return False

In [15]:
def levenshtein(s, t): #расстояние Левенштейна
    if s == t: return 0
    elif len(s) == 0: return len(t)
    elif len(t) == 0: return len(s)
    v0 = [None] * (len(t) + 1)
    v1 = [None] * (len(t) + 1)   
    for i in range(len(v0)):
        v0[i] = i
    for i in range(len(s)):
        v1[0] = i + 1
        for j in range(len(t)):
            cost = 0 if s[i] == t[j] else 1
            v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
        for j in range(len(v0)):
            v0[j] = v1[j]
    return v1[len(t)]

In [16]:
def damerau(s, t): #расстояние Дамерау-Левенштейна (с перестановкой)
    deleteCost = 1
    insertCost = 1
    replaceCost = 1
    transposeCost = 1
    s = " "+s
    t = " "+t
    M = len(s)
    N = len(t)
    d = [list(range(N))]
    for i in range(1,M):
        d.append([])
        for j in range(N):
            d[i].append(0)
        d[i][0] = i
        
    for i in range(1,M):
        for j  in range(1,N):          
            # Стоимость замены
            if (s[i] == t[j]):
                d[i][j] = d[i-1][j-1]
            else:
                d[i][j] = d[i-1][j-1] + replaceCost                   
            d[i][j] = min(
                             d[i][j],                               # замена
                             d[i-1][j] + deleteCost,                # удаление
                             d[i][j-1] + insertCost                 # вставка               
                         )
            if(i > 1 and j > 1 and s[i] == t[j-1] and s[i-1] == t[j]):
                d[i][j] = min(
                                  d[i][j],
                                  d[i-2][j-2] + transposeCost         # транспозиция
                             )
    return d[M-1][N-1]

In [17]:
def GrammaCheck(words): #проверка по грамматике
    #          F I O
    result = [[0,0,0],
              [0,0,0],
              [0,0,0]]

    for i in range(3):
        w = words[i].strip()
        result[i][0] = checkSurnames(w)
        result[i][2] = checkPatronymic(w)

    k = 0
    for i in range(3):
        flag = True
        for j in range(3):
            if (result[i][j] != 0): 
                flag = False 
                break
        if(flag):
            result[i][1] = 1
            k+=1
    if(k==0):
        for i in range(3): result[i][1] = 0.33
    elif(k>1): 
        for i in range(3): result[i][1] = result[i][1]/k
    return result

In [18]:
def checkSurnames(s):#раньше возвращало тру фолс, теперь 1 или 0
    pattern = '\w*(ов|ова|ев|ёв|ева|ёва|ив|ин|ина|ын|их|ых|ский|цкий|ая|ко|дзе|онок|ян)$'
    if (re.match(pattern,s)): return 1
    else: return 0

In [19]:
def checkPatronymic(s):
    pattern = '\w*(ович|евич|ич|овна|евна|ична|инична)$'
    if (re.match(pattern,s)):
        return 1
    else: return 0

In [20]:
#               F     I    O
statistics = [[0.50,0.45,0.05],
              [0.10,0.50,0.40],
              [0.40,0.05,0.55]]
# FIO: 0.50, IOF: 0.40, IFO: 0.15, ElseOptions: 0.05
#пока что совсем от балды, простите
def SetStatistics(order):
    for i in range(3):
        for j in range(3):
            if (j == order[i]): 
                statistics[i][j] += 0.002
            else: statistics[i][j] -= 0.001
#Обновляет статистику с учетом результата выполнения алгоритма            

In [47]:
#я простой блок для проверки методов над матрицами. не смотрите на меня, я тут случайно
    #      F I O
result = [[0,0,1],[0,1,0],[1,0,0]]
words = ["Бучнева", "Анна", "Всеволодовна"]

all_surnames = {"иванов":100,"петров":200,"бугазявкин":5,"соколов":50,"сидоров":150,"сидорок":1 }
all_names = {"иван":100,"петр":200,"петре":1,"кирилл":50,"александр":150,"анатолий":100 }
all_patronymics = {"иванович":100,"петрович":200,"синюевич":5,"кириллович":50,"александрович":150,"анатольевич":100 }

inputText = ["Петров Кирилл Анатольевич",
             "Александр Иванович Сидоров",
             "Бугазявкин Петре Петрович",
             "Синюевич Анатольевич Иван",
             "Иванович Иванов Иван",
             "Сидорок Анатоилй Петрович",
             "Кириллович Иван Соколов"]
print(NotBruteAtAll(inputText))

['Петров Кирилл Анатольевич', 'Сидоров Александр Иванович', 'Бугазявкин Петре Петрович', 'Синюевич Иван Синюевич', 'Иванов Иван Иванович', 'Сидоров Анатолий Петрович', 'Соколов Иван Кириллович']


In [30]:
path_to_names = "D:/NamesFromGufo.pickle"
path_to_patronymics = "D:/PatronymicsFromGufo.pickle"
path_to_new_names = "D:/names_extended.pickle"
path_to_new_surnames = "D:/surnames_extended.pickle"
all_names = []
all_patronymics = []
names_extended = {}
surnames_extended = {}

with open(path_to_names,"rb") as f:
    all_names=pickle.load(f)
with open(path_to_patronymics,"rb") as f:
    all_patronymics=pickle.load(f)    
with open(path_to_new_names,"rb") as f:
    names_extended=pickle.load(f)
with open(path_to_new_surnames,"rb") as f:
    surnames_extended=pickle.load(f)

print(surnames_extended)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
with open("D:/test_changed.pickle","rb") as f:
    tmp=pickle.load(f)

testing_res=NotBruteAtAll(tmp)
for fio in testing_res:
    print(fio)

NameError: name 'all_surnames' is not defined

In [50]:
def NotBruteAtAllOld(tmp): #это будущий общий метод, в котором будут вызываться все следующие
    res = []
    for t in temp:
        result = [] #ответ в шаблоне ФИО
        
        resultStrict = StrictCheck(t)
        if СheckMatrix(resultStrict):
            order = GetOrder(result)
            for i in range(3):
                result.append(words[order[i]])   
        # + установить, что если частота меньше N проверить реплейсом (т.к. только редкие, э-экономия)
        else:
            resultRepl = ReplaceCheck(t)
            resultReplValues = resultRepl.values()
            
            resultGramma = GrammaCheck(t) 
            #Сделать грамма чек если только все остальное пусто 
            # ИЛИ мы сравнили строгий результат и гибкий, и там все непонятно
            #Потому что иначе, если уже найдено, то зачем проверять: если ошибка в написании, она может быть
            resultStatistics = statistics
        #разобраться с Ё и Е
        #result = np.array(result)
        #подсчитываем все с матрицами
        #принимаем решение, записываем в рез
        #вызываем метод SetStatistics и обновляем статистику порядка
        
        res.append(result)
    return res

In [51]:
def NotBruteEnough(tmp):
    res = []

    for t in tmp:
        name = ""
        surname = ""
        patronymic = ""
        t = t.lower()
        words = t.split(" ")

        for w in words:
            w.strip()
            if (name == ""):
                if (w in all_names):
                    name = w

            if (patronymic == ""):
                if (w in all_patronymics):
                    patronymic = w

            if (name != w and patronymic != w and checkSurnames(w) and surname == ""):
                surname = w

        if (name in words): words.remove(name)
        if (patronymic in words): words.remove(patronymic)
        if (surname in words): words.remove(surname)

        if (name == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for n in all_names:
                    dist = levenshtein(w, n)
                    if dist < minDist:
                        minDist = dist
                        name = n.title() #+ " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (patronymic == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for p in all_patronymics:
                    dist = levenshtein(w, p)
                    if dist < minDist:
                        minDist = dist
                        patronymic = p.title() #+ " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (surname == "")& (len(words) > 0):
            surname = words[0]

        name = name.title()
        surname = surname.title()
        patronymic = patronymic.title()
        res.append(surname + " " + name + " " + patronymic)

    return res

In [52]:
#genders Пока не используем
genderTuple = ('male', 'female', 'unknown')

def checkSurnamesGender(s):
    patternMale = '\w*(ов|ев|ий|ын|ин)$'
    patternFem ='\w*(ова|ева|ая|ина|ына)$'
    patternUnknown = '\w*(их|ых|ко|ук|юк|ун|ний|ный|чай|ий|а|ич|ов|ук|ик|ски|ка|ски|цки|дзки)$'
    
    if (re.match(patternMale,s)):
        return genderTuple[0]
    elif (re.match(patternFem,s)):
        return genderTuple[1]
    elif (re.match(patternUnknown,s)):
        return genderTuple[2]
    else: return 'No'

def changeGenderSurname(s):
    patternMaleA = '\w*(ов|ев|ин|ын)$'
    patternFemA ='\w*(ова|ева|ина|ына)$'
    patternMaleB = '\w*(ий)$'
    patternFemB = '\w*(ая)$'
    
    if(re.match(patternMaleA, s)):
        return s + 'a'
    elif (re.match(patternMaleB, s)):       
        s += ' '
        return s.replace('ий ', 'ая')
    elif (re.match(patternFemA, s)):
        return s[0: len(s)-1]
    elif (re.match(patternFemB, s)): 
        s += ' '
        return s.replace('ая ', 'ий')
    else: return s

def checkPatronymicGender(s):
    patternMale = '\w*(ович|евич|ич)$'
    patternFem ='\w*(овна|евна|ична|инична)$'
   
    if (re.match(patternMale,s)):
        return genderTuple[0]
    elif (re.match(patternFem,s)):
        return genderTuple[1]
    else: return 'No'    

def changeGenderPatronymic(s):
    patternMaleA = '\w*(ович|евич)$'
    patternFemA ='\w*(овна|евна)$'
    patternMaleB = '\w*(ич)$'
    patternFemB = '\w*(ична|инична)$'

    if(re.match(patternMaleA, s)):
        s += ' '
        return s.replace('ич ', 'на')
    if(re.match(patternFemA, s)):
        s += ' '
        return s.replace('на ', 'ич')     
    if(re.match(patternMaleB, s)):
        return s + 'на'   
    if(re.match(patternFemB, s)):
        s += ' '
        s = s.replace('инична ', 'ич')
        s = s.replace('ична ', 'ич')
        return s    
    else: return s